<a href="https://colab.research.google.com/github/wahyunh10/Advertising-AB-Testing/blob/main/Advertising_A_B_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Advertising A/B Testing**

The dataset where you can find [here](https://www.kaggle.com/datasets/osuolaleemmanuel/ad-ab-testing) is about advertising agency want to test whether show different type of ad will affecting respond rate of questionnaire that is shared. A/B testing is undertaken to facilitating this. Where a hypothesis is tested and bring a result. You can also found some exploratory data analysis here including seeing trends that happened

**Data Preprocessing**

Import the csv file as panda DataFrames and examine it.

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
import scipy.stats
from math import ceil

A company want to test whether show a creative and interactive ad will increase the amount of questionnaire respond rate that they received

In [ ]:
#Import the excel file and examine it
df = pd.read_csv('AdSmartABdata - AdSmartABdata.csv')
df.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


In [ ]:
df.shape

(8077, 9)

**Cleaning the Dataset:**

Dealing with missing values, correct and create necessary date data types, and drop the duplicate.

In [ ]:
#Investigate missing value
df.isna().sum()

auction_id     0
experiment     0
date           0
hour           0
device_make    0
platform_os    0
browser        0
yes            0
no             0
dtype: int64

In [ ]:
df.dtypes

auction_id     object
experiment     object
date           object
hour            int64
device_make    object
platform_os     int64
browser        object
yes             int64
no              int64
dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8077 entries, 0 to 8076
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   auction_id   8077 non-null   object
 1   experiment   8077 non-null   object
 2   date         8077 non-null   object
 3   hour         8077 non-null   int64 
 4   device_make  8077 non-null   object
 5   platform_os  8077 non-null   int64 
 6   browser      8077 non-null   object
 7   yes          8077 non-null   int64 
 8   no           8077 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 568.0+ KB


In [ ]:
# change date datatype
df['date']=pd.to_datetime(df['date'])

In [ ]:
# check if there's any duplicate record
df.duplicated().sum()

0

In [ ]:
#final DataFrame
df.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


**Exploratory Data Analysis**

In [ ]:
df['experiment'].value_counts()/len(df)*100

control    50.402377
exposed    49.597623
Name: experiment, dtype: float64

There are more people in the control group than the exposed one. However, the gap is not that huge

In [ ]:
df_control_subset=df[df['experiment']=='control']
df_exposed_subset=df[df['experiment']=='exposed']

In [ ]:
df_control_subset[['device_make', 'browser']].describe().transpose()

In control group the most used device by user is generic smartphone, while the most used browser is chrome mobile

In [ ]:
df_exposed_subset[['device_make', 'browser']].describe().transpose()

The same case also happen to exposed group

In [ ]:
df.groupby(by='experiment', as_index=False)[['yes', 'no']].sum()

There 264 people and 322 people who responded yes and no in the questionnaire as the control group. Also, there 308 people and 349 people who responded yes and no as the exposed group. The rest is people who are not respond to the questionnaire

In [ ]:
len(df[(df['yes']==0) & df['no']==0])

To be exact there's 7406 people who didn't respond the questionnaire in both experiment group

**Exploratory Data Analysis : Identifying Trends**

In [ ]:
# break out the value of yes column by time and browser used
by_browser=pd.pivot_table(df,
                               values=['yes'], 
                               index=['date'],
                               columns=['browser'],
                               aggfunc='sum',
                               fill_value=0)
by_browser=by_browser.reset_index()
by_browser

In [ ]:
by_browser.plot(x='date', y='yes', figsize=(8,8))
plt.legend(loc=1)
plt.title('Respondent Who Said Yes : in Date')
plt.show()

As known before, respondent who answer yes use chrome mobile the most. The peak of using happened at 8 July 2020

In [ ]:
# break out the value of yes column by hour and browser used
by_browser1=pd.pivot_table(df,
                               values=['yes'], 
                               index=['hour'],
                               columns=['browser'],
                               aggfunc='sum',
                               fill_value=0)
by_browser1=by_browser1.reset_index()
by_browser1

In [ ]:
by_browser1.plot(x='hour', y='yes', figsize=(8,8))
plt.legend(loc=1)
plt.title('Respondent Who Said Yes : in Hours')
plt.show()

Some browsers mostly used at 15 o'clock by respondent who answer yes

In [ ]:
# break out the value of no column by date and browser used
by_browserno=pd.pivot_table(df,
                               values=['no'], 
                               index=['date'],
                               columns=['browser'],
                               aggfunc='sum',
                               fill_value=0)
by_browserno=by_browserno.reset_index()
by_browserno

In [ ]:
by_browserno.plot(x='date', y='no', figsize=(8,8))
plt.legend(loc=1)
plt.title('Respondent Who Said No : in Date')
plt.show()

While respondent who said no mostly use chrome mobile at 9 July 2020

In [ ]:
# break out the value of no column by hour and browser used
by_browserno1=pd.pivot_table(df,
                               values=['no'], 
                               index=['hour'],
                               columns=['browser'],
                               aggfunc='sum',
                               fill_value=0)
by_browserno1=by_browserno1.reset_index()
by_browserno1

In [ ]:
by_browserno1.plot(x='hour', y='no', figsize=(8,8))
plt.legend(loc=1)
plt.title('Respondent Who Said No : in Hours')
plt.show()